In [ ]:
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 12)

import ipywidgets
from ipywidgets import interact

import numpy as np
#np.set_printoptions(threshold=np.nan)
from matplotlib import pyplot as plt
from matplotlib import cm

import ctapipe
import ctapipe.visualization
from ctapipe.io.hessio import hessio_event_source
import pyhessio

#ctapipe.version.get_version()
print(ctapipe.__version__)
print(pyhessio.__version__)

from ctapipe.instrument import CameraGeometry

In [ ]:
# calibrator
from ctapipe.calib import CameraCalibrator

calib = CameraCalibrator(None, None)

In [ ]:
tel_id = 1
channel = 0

#file_path = "/Users/jdecock/vm-shared/data/Prod3b/LaPalma/gamma_20deg_0deg_run104___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz"
file_path = "/Users/jdecock/vm-shared/data/Prod3b/LaPalma/proton_20deg_0deg_run50001___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz"

source = hessio_event_source(file_path,
                             allowed_tels=[tel_id])

In [ ]:
event_dict = {}

NUM_MAX_IMAGES = 100

for ev in source:
    calib.calibrate(ev)  # calibrate the event

    # GET IMAGES ##############################################

    channel = 0

    pe = ev.mc.tel[tel_id].photo_electron_image
    pedestal = ev.mc.tel[tel_id].pedestal[channel]
    gain = ev.mc.tel[tel_id].dc_to_pe[channel]
    uncalibrated_image = ev.r0.tel[tel_id].adc_sums[channel]

    calibrated_image = ev.dl1.tel[tel_id].image[channel]

    ## Mix channels (not done in ctapipe because cameras will do it automatically in production, there will be only 1 channel then ; i.e. this issue only concerns simulations)
    ## Taken from TinoCTA ./modules/prepare_event.py l.77
    #pmt_signal = event.dl1.tel[tel_id].image
    #if pmt_signal.ndim > 1 and pmt_signal.shape[0] > 1:
    #    pick = (pmt_signal > 14).any(axis=0) != np_true_false
    #    pmt_signal = pmt_signal.T[pick.T]
    #else:
    #    pmt_signal = pmt_signal.ravel()

    #threshold = 190  # cf. "calib_find_channel_selection_threshold" notebook
    #calibrated_image[1, calibrated_image[0,:] <= threshold] = 0
    #calibrated_image[0, calibrated_image[0,:] >  threshold] = 0
    #calibrated_image = calibrated_image.sum(axis=0)

    pixel_pos = ev.inst.pixel_pos[tel_id]

    event_id = ev.dl0.event_id

    foclen = ev.inst.optical_foclen[tel_id]

    num_channels = len(ev.r0.tel[tel_id].adc_sums)

    event_dict[int(ev.dl0.event_id)] = {
        "pedestal": pedestal,
        "gain": gain,
        "adc_sum": uncalibrated_image,
        "pe": pe,
        "calibrated_image": calibrated_image,
        "num_channels": num_channels,
        "geom_x": pixel_pos[0],
        "geom_y": pixel_pos[1],
        "foclen": foclen
    }
    
    if len(event_dict) > NUM_MAX_IMAGES:
        break

In [ ]:
@interact(event_id=list(event_dict.keys()), image_type=["pedestal", "gain", "adc_sum", "calibrated_image", "pe"])
def gui(event_id, image_type):
    ev = event_dict[int(event_id)]

    #########################################################################

    x, y = ev["geom_x"], ev["geom_y"]
    foclen = ev["foclen"]
    geom = ctapipe.instrument.CameraGeometry.guess(x, y, foclen)

    disp = ctapipe.visualization.CameraDisplay(geom, title='CT%d' % tel_id)
    disp.enable_pixel_picker()
    disp.add_colorbar()

    disp.axes.set_title('Telescope {:03d}'.format(tel_id))

    # DISPLAY TIME-VARYING EVENT ############################################

    #data = event.dl0.tel[tel_num].adc_samples[channel]
    #for ii in range(data.shape[1]):
    #    disp.image = data[:, ii]
    #    disp.set_limits_percent(70)   # TODO
    #    plt.savefig('CT{:03d}_EV{:05d}_S{:02d}.png'.format(tel_num, event_id, ii), bbox_inches='tight')

    # DISPLAY INTEGRATED EVENT ##############################################

    disp.image = ev[image_type]

    #disp.set_limits_minmax(0, 9000)
    disp.set_limits_percent(70)        # TODO

    # PLOT ##################################################################

    plt.show()